**подключение библиотек**   
transforms - для быстрого умнлжения на матрицу Адамара

In [232]:
import numpy as np # type: ignore
import sys
import os

from scipy.linalg import qr # type: ignore
from scipy.linalg import svd # type: ignore
import numpy as np # type: ignore
import transforms # type: ignore

import time
import random

print(sys.executable)

/home/taisia/anaconda3/envs/rand_dec/bin/python


**my_matmul**
рукописное умножение для сравнений

In [233]:
def my_matmul(A, B):
  n, m = A.shape
  _, k = B.shape
  C = np.zeros((n, k))
  for i in range(0, n):
    for j in range(0, k):
      for l in range(0, m):
        C[i][j] += A[i][l] * B[l][j]
  return C

In [234]:
def matmul(A, B):
  return A @ B

**transforms_projection**  
быстрое умножение матрицы.  
**mul_mode** влияет на способ умножения:
- fast_hadamar_transform
- fft

In [235]:
def transforms_projection(A, k, mul_mode, p=5):
    m, n = A.shape
    l = k + p
    Y = A.copy()
    if mul_mode == "hadamar":
      for i in range(0, m):
        Y[i, :] = transforms.fast_hadamard_transform(Y[i, :])
    else:
      for i in range(0, m):
        Y[i, :] = transforms.fft(Y[i, :])
    random_cols = np.random.choice(n, l, replace = False)
    Y = Y[:, random_cols]
    return Y


**get_QR**  
получение матрцы Q

In [236]:
def get_QR(A, Y0, q=3):
    Q0, _ = np.linalg.qr(Y0)
    Qj = Q0
    for _ in range(q):
        Y_tilde_j = A.conj().T @ Qj
        Q_tilde_j, _ = np.linalg.qr(Y_tilde_j)

        Yj = A @ Q_tilde_j
        Qj, _ = np.linalg.qr(Yj)

    return Qj

## Далее 3 функции описывают 3 разных подхода умножения матриц в алгоритме 4.4

- **randomized_subspace_iteration_with_hadamar**  
Алгоритм 4.4 из статьи.  
но для получения Y0 = A*Ω и самой случайной матрицы Ω используем transforms_projection with hadamar_mode=True

In [237]:
def randomized_subspace_iteration_with_hadamar(A, k, p=3, q=3):
    
    Y0 = transforms_projection(A, k, "hadamar", p)
    return get_QR(A, Y0, q)


- **randomized_subspace_iteration_with_fft**  
Честный алгоритм 4.4, использует fft умножение

In [238]:
def randomized_subspace_iteration_with_fft(A, k, p, q=3):

    Y0 = transforms_projection(A, k, "fft", p)
    return get_QR(A, Y0, q)


**randomized_subspace_iteration_with_built_in**  
Алгоритм 4.4 с встроенным умножением

In [239]:
def randomized_subspace_iteration_with_built_in(A, k, p, q=3):
    
    _, n = A.shape
    l = k + p
    Omega = np.random.randn(n, l)
    Y0 = A @ Omega

    return get_QR(A, Y0, q)

### 2 функции подсчета loss  
1) использует встроенный mat_mul, честно считает ошибку
2) использует несколько случайных проекций. Возвращает максимальную ошибку.

- **get_loss**  
возвращает нормированную ошибку Q\*QT\*A - A.  
считает через встроенное умножение

In [240]:
def get_loss(A, Q):
  return np.linalg.norm(Q @ (Q.T @ A) - A) / np.linalg.norm(A)

In [241]:
accuracy = 1e-8

- **get_loss_with_random_projections**  
    - get_array_projections.  
    первый шаг - формирование проекций
    - get_loss_with_random_projections.  
    второй шаг - возврат максимальной ошибки

In [242]:
def get_array_projections(A, seed=10):
    _, n = A.shape
    projections = list()
    for _ in range(seed):
        rand_vec = np.random.randn(n)
        projection = A @ rand_vec
        projections.append(projection / np.linalg.norm(projection))
    return projections

In [243]:
def get_loss_with_random_projections(A, projections_arr, Q):
    max_loss = 0
    for projection in projections_arr:
        result = Q @ (Q.T @ projection)
        if np.linalg.norm(result - projection) > max_loss:
            max_loss = np.linalg.norm(result - projection)
    return max_loss
########## !!!!!!!!!!!!!!!!!!!!!!

In [244]:
def check_loss_rand(A, get_Q_func, projections_arr, k, p=3, q=3, max_loss = accuracy):
    Q = get_Q_func(A, k, p, q)
    loss = get_loss_with_random_projections(A, projections_arr, Q)
    return loss < max_loss

In [245]:
def check_loss_k(A, get_Q_func, k, p=3, q=3, max_loss = accuracy):
    Q = get_Q_func(A, k, p, q)
    loss = get_loss(A, Q)
    return loss < max_loss

In [246]:
def get_rank_binary_search(A, left, right, get_Q_func, p=5, q=3, max_loss = accuracy):
  while right - left > 1: 
    k = (right + left) // 2
    if (check_loss_k(A, get_Q_func, k, p, q, max_loss)):
      right = k
    else:
      left = k
  return right

**get_project_rank**  
наш поиск ранга. 
1) ищет первую степень двойки, для которой ранг точно хорошо аппроксимирует 
2) бинарным поиском находим ранг  
проверка идет по check_loss_k.

In [247]:
def get_project_rank(A, get_Q_func, p=5, q=3, debug=False, max_loss=accuracy):
  k = 1
  while True:
    Q = get_Q_func(A, k, p, q)
    loss = get_loss(A, Q)
    if debug:
      print(f'now k = {k}, now loss = {loss}')
      
    if (check_loss_k(A, get_Q_func, k, p, q, max_loss)):
      return get_rank_binary_search(A, k // 2, k, get_Q_func, p, q, max_loss)
    k *= 2


**get_project_rank_with_random_projections**  
такой же поиск ранга, но с случайными проекциями

In [248]:
def get_project_rank_with_random_projections(A, get_Q_func, p=5, q=3, seed=10, debug=False, max_loss=accuracy):
    projections_arr = get_array_projections(A, seed)
    k = 1
    while True:
        Q = get_Q_func(A, k, p, q)
        loss = get_loss(A, Q)
        if debug:
            print(f'now k = {k}, now loss = {loss}')
        if (check_loss_rand(A, get_Q_func, projections_arr, k, p, q, max_loss)):
            return get_rank_binary_search(A, k // 2, k, get_Q_func, p, q, max_loss)
        k *= 2

# -------------
# **далее тесты**
# --------------

In [249]:
def make_random_rank_k_matrix(m, n, k, random_state=None):
    B = np.random.randn(m, k)
    C = np.random.randn(k, n)
    A: np.matrix = B @ C
    return A

In [250]:
k = 45
p = 5
q = 3

In [251]:
m, n = 2**11, 2**11

In [252]:
A = make_random_rank_k_matrix(m, n, k)

### 3 функции запуска и получения времени  
1) get_project_time - наш алгоритм с бин поиском
2) get_project_time_with_random_projections - наш алгоритм с бин поиском и случайными прекциями 
3) get_classic_time - встроенный ранг

In [253]:
def get_project_time(A, iteration):
    time_start = time.time()
    rank = get_project_rank(A, iteration)
    time_finish = time.time()
    return (time_finish - time_start, rank)

In [254]:
def get_project_time_with_random_projections(A, iteration, seed=10):
    time_start = time.time()
    rank = get_project_rank_with_random_projections(A, iteration, seed)
    time_finish = time.time()
    return (time_finish - time_start, rank)

In [255]:
def get_classic_time(A):
    time_start = time.time()
    rank = np.linalg.matrix_rank(A)
    time_finish = time.time()
    return (time_finish - time_start, rank)

**Хотим тестировать:**
1) умножение матрицы. будем передавать используемую функцию
    - randomized_subspace_iteration_**with_hadamar**
    - randomized_subspace_iteration_**with_fft**
    - randomized_subspace_iteration_**with_built_in**
2) проверку ошибки:
    - честную
    - со случайными проекциями.  
на проверке ошибки так же используем то же умножение


In [256]:
from tqdm import tqdm

### hadamar

In [264]:
haramar_time = get_project_time(A, randomized_subspace_iteration_with_hadamar)
haramar_time_with_projections = get_project_time_with_random_projections(A, randomized_subspace_iteration_with_hadamar)

print(f'rank time with hadamar')
print(f'without projections\n{haramar_time}')
print(f'with projections\n{haramar_time_with_projections}')

rank time with hadamar
without projections
(3.291602611541748, 40)
with projections
(2.392115831375122, 35)


### fft

In [263]:
fft_time = get_project_time(A, randomized_subspace_iteration_with_fft)
fft_time_witch_projections = get_project_time_with_random_projections(A, randomized_subspace_iteration_with_fft)

print(f'rank time with fft.')
print(f'without projections\n{fft_time}')
print(f'with projections\n{fft_time_witch_projections}')

rank time with fft.
without projections
(5.898900032043457, 40)
with projections
(5.70976185798645, 35)


### built-in

In [265]:
built_in_time = get_project_time(A, randomized_subspace_iteration_with_built_in)
built_in_time_witch_projections = get_project_time_with_random_projections(A, randomized_subspace_iteration_with_built_in)

print(f'rank time with built_in.')
print(f'without projections\n{built_in_time}')
print(f'with projections\n{built_in_time_witch_projections}')

rank time with built_in.
without projections
(2.0759363174438477, 40)
with projections
(1.8638813495635986, 35)


In [260]:
# from tqdm import tqdm

# all_time_subspace_rand = 0
# all_time_classic = 0
# for i in tqdm(range(5), desc="running"):
    
#     all_time_classic += get_classic_time(A)[0]
#     all_time_subspace_rand += get_project_time_with_random_projections(A, randomized_subspace_iteration)[0]
    
# print(f"classic rank = {all_time_classic}")
# print (f"our fast rank = {all_time_subspace_rand}")
